<a href="https://colab.research.google.com/github/samp3209/wineproject/blob/main/winemodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [11]:
df = pd.read_csv('wine - final (1).csv')

In [12]:
df

,pp750ml,wine_names,year,price,region,country,quantity
0,4.16,"Al-Ria White 2022, Algarve, Portugal",2022,4.16,algarve,port,750 ml
1,13.04,"Fiano di Avellino DOCG 2022, Feudi di San Greg...",2022,13.04,campania,italy,750 ml
2,398.39,2022 Egon Muller Wiltinger Braune Kupp Rieslin...,2022,398.39,masel,germany,750 ml
3,383.91,2022 Egon Muller Wiltinger Braune Kupp Rieslin...,2022,383.91,masel,germany,750 ml
4,319.38,2022 Egon Muller Scharzhof Riesling Mosel,2022,319.38,masel,germany,750 ml
...,...,...,...,...,...,...,...
9225,2544.43,"Clos des Lambrays 1937, Dom. des Lambrays, Cot...",1937,2544.43,burgundy,france,750 ml
9226,737.72,1924 Kopke Vintage Port 1924,1924,737.72,duoro,port,750 ml
9227,5000.00,"[lot 352], Chateau Lafite Rothschild",1914,5000.00,bordeaux,france,750 ml
9228,6995.00,1914 Moet & Chandon Vintage Champagne 750ml,1914,6995.00,champagne,france,750 ml


In [13]:
# one hot encode the region and country columns
region_dummies = pd.get_dummies(df['region'], prefix='region')
country_dummies = pd.get_dummies(df['country'], prefix='country')

# concatenate the original dataframe with the one hot encoded columns
df = pd.concat([df, region_dummies, country_dummies], axis=1)

# drop the original region and country columns
df = df.drop(['region', 'country', 'price'], axis=1)

# show the resulting dataframe
print(df)

      pp750ml                                         wine_names  year  \
0        4.16               Al-Ria White 2022, Algarve, Portugal  2022   
1       13.04  Fiano di Avellino DOCG 2022, Feudi di San Greg...  2022   
2      398.39  2022 Egon Muller Wiltinger Braune Kupp Rieslin...  2022   
3      383.91  2022 Egon Muller Wiltinger Braune Kupp Rieslin...  2022   
4      319.38          2022 Egon Muller Scharzhof Riesling Mosel  2022   
...       ...                                                ...   ...   
9225  2544.43  Clos des Lambrays 1937, Dom. des Lambrays, Cot...  1937   
9226   737.72                       1924 Kopke Vintage Port 1924  1924   
9227  5000.00               [lot 352], Chateau Lafite Rothschild  1914   
9228  6995.00        1914 Moet & Chandon Vintage Champagne 750ml  1914   
9229  6995.00        1911 Moet & Chandon Vintage Champagne 750ml  1911   

     quantity  region_Rheinhessen  region_algarve  region_alsace  \
0      750 ml                   0          

In [31]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from fuzzywuzzy import fuzz

# example wine names
wine_names = ['Chateau Margaux', 'Chateau Margeaux', 'Margaux Chateau', 'Margeaux']

# compute pairwise Levenshtein distance
for i in range(len(wine_names)):
    for j in range(i+1, len(wine_names)):
        print(wine_names[i], wine_names[j], fuzz.ratio(wine_names[i], wine_names[j]))

Chateau Margaux Chateau Margeaux 97
Chateau Margaux Margaux Chateau 47
Chateau Margaux Margeaux 61
Chateau Margeaux Margaux Chateau 45
Chateau Margeaux Margeaux 67
Margaux Chateau Margeaux 61


/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
from fuzzywuzzy import process

# example wine names
wine_names = df['wine_names']

# create a dictionary of common names based on similarity
common_names = {}
for wine_name in wine_names:
    match = process.extractOne(wine_name, common_names.keys(), scorer=fuzz.ratio)
    if match is not None and match[1] >= 90:  # adjust similarity threshold as needed
        common_names[match[0]].append(wine_name)
    else:
        common_names[wine_name] = [wine_name]

# assign common names to each wine
for i, wine_name in enumerate(wine_names):
    for common_name, wines in common_names.items():
        if wine_name in wines:
            wine_names[i] = common_name
            break

print(wine_names)

In [18]:
x= df['pp750ml']
y = df.loc[:, ~df.columns.isin(['pp750ml', 'wine_names'])]

In [23]:
from sklearn.model_selection import train_test_split
import numpy as np 
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [26]:
df['wine_names'].nunique()

7492

In [24]:
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)

ValueError: ignored

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Initialize the model
model = DecisionTreeRegressor()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
model = RandomForestRegressor()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)

In [ ]:
from sklearn.svm import SVR

# Initialize the model
model = SVR()

# Fit the model to your data
model.fit(X_train, y_train)

# Make predictions on new data
y_pred = model.predict(X_test)